<a href="https://colab.research.google.com/github/huavanson/CS114.K21/blob/master/sarcasm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"sonhuavan","key":"899cfa81c0f3f4051aee96eb7e74b10f"}'}

Sau khi up file kaggle.json lên thì tạo 1 thư mục có tên .kaggle đưa file vừa up vào 

In [ ]:
!ls -lha kaggle.json
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

-rw-r--r-- 1 root root 65 May  8 07:25 kaggle.json


Bước này Download dataset trên kaggle về 


In [3]:
!kaggle datasets download -d rmisra/news-headlines-dataset-for-sarcasm-detection

  0% 0.00/3.30M [00:00<?, ?B/s]
100% 3.30M/3.30M [00:00<00:00, 110MB/s]


Check xem các thư viện cần thiết đã được install vào chưa 


In [4]:
!pip install numpy
!pip install pandas 
!pip install sklearn

In [5]:
import json
import pandas as pd, numpy as np, re, time
from nltk.stem.porter import PorterStemmer
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.svm import LinearSVC
from sklearn.model_selection import cross_val_score
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier

In [6]:
!unzip news-headlines-dataset-for-sarcasm-detection.zip

Archive:  news-headlines-dataset-for-sarcasm-detection.zip
  inflating: Sarcasm_Headlines_Dataset.json  
  inflating: Sarcasm_Headlines_Dataset_v2.json  


Đọc dataset bằng Pandas và check số lượng data 

In [26]:
df=pd.read_json('Sarcasm_Headlines_Dataset.json',lines=True)
# shape and head
df.head()

,article_link,headline,is_sarcastic
0,https://www.huffingtonpost.com/entry/versace-b...,former versace store clerk sues over secret 'b...,0
1,https://www.huffingtonpost.com/entry/roseanne-...,the 'roseanne' revival catches up to our thorn...,0
2,https://local.theonion.com/mom-starting-to-fea...,mom starting to fear son's web series closest ...,1
3,https://politics.theonion.com/boehner-just-wan...,"boehner just wants wife to listen, not come up...",1
4,https://www.huffingtonpost.com/entry/jk-rowlin...,j.k. rowling wishes snape happy birthday in th...,0


In [9]:
df.shape

(26709, 3)

check xem data có bị missing value hay không

In [25]:
print(df.isnull().any(axis = 0))

article_link    False
headline        False
is_sarcastic    False
dtype: bool


Column article_link không cần thiết để detect 1 headline có sarcasm hay không nên có thể được lượt bỏ 


In [40]:
df = df.drop(['article_link'], axis=1)

Dữ liệu không bị missing value nhưng cột tiêu đề nên được loại bỏ các kí tự không cần thiết vì vậy em sử dụng Regular Expression ( Biểu thức chính quy) để loại bỏ các kí tự đặc biệt đó ví dụ dấu nháy dấu chấm v.v

In [29]:
df['headline'] = df['headline'].apply(lambda s : re.sub('[^a-zA-Z]', ' ', s))


In [41]:
df.head(5)

,headline,is_sarcastic
0,former versace store clerk sues over secret b...,0
1,the roseanne revival catches up to our thorn...,0
2,mom starting to fear son s web series closest ...,1
3,boehner just wants wife to listen not come up...,1
4,j k rowling wishes snape happy birthday in th...,0


In [22]:
features = df['headline']
labels = df['is_sarcastic']
features_compare=features


Xử dụng PorterStemmer để cắt giảm 1 từ thành từ gốc của nó 
Ví dụ : "Reading" or "Reader"
Sau khi stemming sẽ trở thành "Read" ( cho cả 2 từ trên)

In [23]:
ps = PorterStemmer()
features = features.apply(lambda x: x.split())
features = features.apply(lambda x : ''.join([ps.stem(word) for word in x]))

So sánh chuỗi trước và sau khi stemming 

In [24]:
print(features_compare[0])
print(features[0])

former versace store clerk sues over secret  black code  for minority shoppers
formerversacstoreclerksueoversecretblackcodeforminorshopper


Sử dụng tfidf để chuyển list features về 1 dạng ma trận

In [34]:
from sklearn.feature_extraction.text import TfidfVectorizer
tv = TfidfVectorizer(max_features = 10000)
features = list(features)
features = tv.fit_transform(features).toarray()

features sau khi được chuyển sang ma trận 


In [36]:
print(features)

[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


In [37]:
features_train, features_test, labels_train, labels_test = train_test_split(features, labels, test_size = 0.2, random_state = 0)

In [38]:
# Using linear support vector classifier
lsvc = LinearSVC()
# training the model
lsvc.fit(features_train, labels_train)
# getting the score of train and test data
print(lsvc.score(features_train, labels_train)) 
print(lsvc.score(features_test,labels_test))


0.740721673608836
0.566828903032572
